In [26]:
import pathlib
from collections import namedtuple as NT
from scipy.io import loadmat
import numpy as np
import xarray as xr
import csv
π = np.pi

In [2]:
model_path = pathlib.Path('./Sky_Light_Reflection/model1.mat')

In [27]:
denv_vars = ['wind', 'od', 'C', 'zen_sun', 'wtem', 'sal']

def unpack_env_rho(model_path):
    model1 = loadmat(model_path)
    env, rho = model1['env'], model1['rho']
    d_env = dict.fromkeys(denv_vars)
    d_rho = dict.fromkeys(['sky', 'sun', 'sca2vec', 'rho'])
    d_env['wind'] = env['wind'][0][0].squeeze()
    d_env['od'] = env['od'][0][0].squeeze()
    d_env['C'] = env['C'][0][0].squeeze()
    d_env['zen_sun'] = env['zen_sun'][0][0].squeeze()
    d_env['wtem'] = env['wtem'][0][0].squeeze()
    d_env['sal'] = env['sal'][0][0].squeeze()
    d_rho['sky'] = rho['sky'][0][0].squeeze()
    d_rho['sun'] = rho['sun'][0][0].squeeze()
    d_rho['sca2vec'] = rho['sca2vec'][0][0].squeeze()
    d_rho['rho'] = rho['rho'][0][0].squeeze()
    return d_env, d_rho

In [25]:
type(model_path)

pathlib.PosixPath

In [22]:
denv

{'wind': array(10, dtype=uint8),
 'od': array(0.13),
 'C': array(100, dtype=uint8),
 'zen_sun': array(30, dtype=uint8),
 'wtem': array(15, dtype=uint8),
 'sal': array(35, dtype=uint8)}

In [38]:
denv.values()

dict_values([array(10, dtype=uint8), array(0.13), array(100, dtype=uint8), array(30, dtype=uint8), array(15, dtype=uint8), array(35, dtype=uint8)])

In [40]:
d = [val for val in denv.values()]

In [122]:
with open('denv.csv') as f:
    rows = f.readlines()
    d = {k:v for k,v in zip(rows[0].strip().split(','),
                            np.array(rows[1].strip().split(','), dtype='f2'))}

In [123]:
d

{'wind': 10.0,
 'od': 0.13,
 'C': 100.0,
 'zen_sun': 30.0,
 'wtem': 15.0,
 'sal': 35.0}

In [98]:
for i, row in rows:
    data = row.strip().split(',')
    if i>0: data = np.array(data)


['wind', 'od', 'C', 'zen_sun', 'wtem', 'sal']
['10', '0.13', '100', '30', '15', '35']


In [89]:
f = open'denv.csv')
reader = csv.reader(f)
for row in reader:
    print(row)
    print(row[0])
    print(row[0],row[1],)

['wind', 'od', 'C', 'zen_sun', 'wtem', 'sal']
wind
wind od
['10', '0.13', '100', '30', '15', '35']
10
10 0.13


In [4]:
denv, dro = unpack_env_rho(model_path)

In [5]:
sensor = dict.fromkeys(['ang', 'wv', 'ang2'])
sensor['ang'] = np.array([40, 45])
sensor['wv'] = np.arange(350, 1010, 10)
sensor['ang2'] = sensor['ang'] + np.array([0, 180])

In [6]:
sensor['pol'] = np.deg2rad(sensor['ang'])

In [9]:
def my_sph2cart(azm, zen, r):
    def sph2cart(azm, elev, r):
        cos_elev = np.cos(elev)
        x = r * cos_elev * np.cos(azm)
        y = r * cos_elev * np.sin(azm)
        z = r * np.sin(elev)
        return x, y, z
    x, y, z = sph2cart(azm, π/2 - zen, r)
    return np.c_[x.ravel(), y.ravel(), z.ravel()].squeeze()

In [10]:
sensor['vec'] = my_sph2cart(sensor['pol'][1], sensor['pol'][0], 1)

In [11]:
sensor['pol2'] = np.deg2rad(sensor['ang2'])

In [12]:
project_dir = pathlib.Path.home() / 'PROJECTS/DAURIN/Sky_Light_Reflection/'
db_path = project_dir / 'db.mat'

In [102]:
def find_quads(zen, azm):
    #zen, azm = sensor_pol
    print(f'zen: {zen}')
    print(azm)
    loc = None
    try:
        with xr.open_dataset(db_path, group='quads') as quads:
            tmp = np.sqrt((quads.zen[:] - zen)**2 + (quads.azm[:] - azm)**2)
            loc = np.argmin(tmp)
    except:
        pass
    finally:
        return loc

In [103]:
%%time
sensor['loc2'] = find_quads(*sensor['pol2'])

zen: 0.6981317007977318
3.9269908169872414
CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 31.3 ms


* \\(\rightarrow\\) my_sph2cart
* \\(\rightarrow\\) find_quads
* \\(\rightarrow\\) get_prob
    *  \\(\rightarrow\\) sky_light_reflection2
        * \\(\rightarrow\\) gen_vec_polar
        * \\(\rightarrow\\) prob_reflection
            * \\(\rightarrow\\) vec_length
            * \\(\rightarrow\\) my_cart2sph
            * \\(\rightarrow\\) cox_munk
            * \\(\rightarrow\\) rayleighcdf
        * \\(\rightarrow\\) gen_vec_quad
    

In [171]:
# get prob

with xr.open_dataset(db_path, group='quads') as quads:
    prob = quads.zen
    #angr_sky 
    # sky_light_reflection2
    # polar quad, 1st in quads
    zen0 = quads.zen.data[0][0]
    # generate sky vector
    #p_vec = gen_vec_polar(zen0, quads.sun05, 100)
    sun05 = quads.sun05.data.squeeze()

In [185]:
# sky_light_reflection2
# gen_vec_polar
def gen_vec_polar(zen, sun05, num=10):
    ϕ = np.linspace(0, 2*π, 100)
    sin_sun05 = np.sin(sun05)
    tmp = np.c_[sin_sun05*np.cos(ϕ), sin_sun05 * np.sin(ϕ), np.cos(sun05)*np.ones_like(ϕ)]
    tmp = np.insert(tmp, 0, [0, 0, 1], axis=0)
    ry = np.c_[[np.cos(zen), 0, np.sin(zen)], 
               [0, 1, 0], 
               [-np.sin(zen), 0, np.cos(zen)]]
    vec = tmp @ ry
    return vec    

pvec = gen_vec_polar(zen0, sun05, num=100)


In [226]:
# prob_reflection
# def prob_reflection(inc, refl, wind):
inc = -pvec
refl = sensor['vec']
wind = denv['wind']
n = refl - inc  
n /= np.sqrt(np.sum(np.power(np.abs(n), 2), axis=1))[:,None]

In [225]:
# my_cart2sph

array([[0.24184476, 0.24184476, 0.93969262],
       [0.24417634, 0.24170057, 0.93912659],
       [0.24416238, 0.2418487 , 0.93909208],
       [0.2441391 , 0.24199682, 0.93905998],
       [0.24410659, 0.24214431, 0.93903041],
       [0.24406499, 0.24229059, 0.93900349],
       [0.24401446, 0.24243507, 0.93897933],
       [0.2439552 , 0.24257717, 0.93895803],
       [0.24388745, 0.24271632, 0.93893967],
       [0.24381149, 0.24285196, 0.93892432],
       [0.24372763, 0.24298355, 0.93891205],
       [0.24363619, 0.24311055, 0.93890291],
       [0.24353754, 0.24323246, 0.93889692],
       [0.24343209, 0.24334879, 0.93889413],
       [0.24332026, 0.24345907, 0.93889453],
       [0.24320249, 0.24356286, 0.93889812],
       [0.24307926, 0.24365974, 0.9389049 ],
       [0.24295106, 0.24374932, 0.93891483],
       [0.24281841, 0.24383124, 0.93892787],
       [0.24268184, 0.24390517, 0.93894398],
       [0.2425419 , 0.24397082, 0.93896308],
       [0.24239915, 0.24402792, 0.93898511],
       [0.

In [218]:
pvec[:2]

array([[0.        , 0.        , 1.        ],
       [0.0046557 , 0.        , 0.99998916]])

In [210]:
refl

array([0.45451948, 0.45451948, 0.76604444])

In [221]:
(refl + pvec)[:2]

array([[0.45451948, 0.45451948, 1.76604444],
       [0.45917518, 0.45451948, 1.76603361]])

In [219]:
n[:2]

array([[0.45451948, 0.45451948, 1.76604444],
       [0.45917518, 0.45451948, 1.76603361]])

In [204]:
wind

array(10, dtype=uint8)

In [205]:
refl - wind

array([-9.54548052, -9.54548052, -9.23395556])